# Asset Selection

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from sklearn import covariance

import datetime as dt
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

import pickle as pk
import requests

import random

jtplot.style(figsize=(15, 9))

In [3]:
# import bs4 as bs

# response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# soup = bs.BeautifulSoup(response.text)
# table = soup.find('table', {'class': 'wikitable sortable'})

# tickers = []
# for row in table.findAll('tr')[1:]:
#     ticker = row.findAll('td')[0].text.replace('\n', '')
#     if '.' in ticker:
#         ticker = ticker.replace('.', '-')
#     tickers.append(ticker)

# with open('../data/sp500tickers.pickle', 'wb') as f:
#     pk.dump(tickers, f)

# print("---------------")
# print("Ticker Amount: {}".format(len(tickers)))
# print("---------------")

---------------
Ticker Amount: 503
---------------


In [4]:
with open('../data/sp500tickers.pickle', 'rb') as f:
    sp500 = pk.load(f)

with open('../../../alphavantage_token.TXT', 'rb') as f:
    TOKEN = f.read().decode('utf-8')

In [5]:
sp500[:10]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

In [7]:
d = {'Symbol': 'MMM', 'AssetType': 'Common Stock', 'Name': '3M Company',
     'Description': 'ood, a suburb of Saint Paul, Minnesota.',
     'CIK': '66740', 'Exchange': 'NYSE', 'Currency': 'USD',
     'Country': 'USA', 'Sector': 'LIFE SCIENCES',
     'Industry': 'SURGICAL & MEDICAL INSTRUMENTS & APPARATUS',
     'Address': '3M CENTER, BLDG. 220-13E-26A, ST PAUL, MN, US',
     'FiscalYearEnd': 'December', 'LatestQuarter': '2024-03-31',
     'MarketCapitalization': '57903698000', 'EBITDA': '8146000000',
     'PERatio': 'None', 'PEGRatio': '1.905', 'BookValue': '8.79',
     'DividendPerShare': '6.01', 'DividendYield': '0.0268',
     'EPS': '-12.73', 'RevenuePerShareTTM': '58.89',
     'ProfitMargin': '-0.216', 'OperatingMarginTTM': '0.217',
     'ReturnOnAssetsTTM': '0.0758', 'ReturnOnEquityTTM': '-0.693',
     'RevenueTTM': '32653001000', 'GrossProfitTTM': '15000000000',
     'DilutedEPSTTM': '-12.73', 'QuarterlyEarningsGrowthYOY': '-0.051',
     'QuarterlyRevenueGrowthYOY': '-0.003', 'AnalystTargetPrice': '110',
     'AnalystRatingStrongBuy': '0', 'AnalystRatingBuy': '5',
     'AnalystRatingHold': '11', 'AnalystRatingSell': '2',
     'AnalystRatingStrongSell': '0', 'TrailingPE': '-',
     'ForwardPE': '10.6', 'PriceToSalesRatioTTM': '1.546',
     'PriceToBookRatio': '10.51', 'EVToRevenue': '1.879',
     'EVToEBITDA': '-9.08', 'Beta': '1.001', '52WeekHigh': '105.31',
     '52WeekLow': '68.63', '50DayMovingAverage': '101.16',
     '200DayMovingAverage': '97.91', 'SharesOutstanding': '553361000',
     'DividendDate': '2024-06-12', 'ExDividendDate': '2024-05-23'}
d.keys()

dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'AnalystRatingStrongBuy', 'AnalystRatingBuy', 'AnalystRatingHold', 'AnalystRatingSell', 'AnalystRatingStrongSell', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'DividendDate', 'ExDividendDate'])

In [27]:
data = pd.read_csv('../data/sp500_Company_Overview.csv', index_col=0)
data.head()

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740.0,NYSE,USD,USA,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,...,1.879,-9.08,1.001,105.52,68.63,101.29,97.98,5.533610e+08,2024-06-12,2024-05-23
1,ABT,Common Stock,Abbott Laboratories,Abbott Laboratories is an American multination...,1800.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,...,4.928,18.75,0.718,120.40,88.33,103.94,106.53,1.739630e+09,2024-08-15,2024-07-15
2,ABBV,Common Stock,AbbVie Inc,AbbVie is an American publicly traded biopharm...,1551152.0,NYSE,USD,USA,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,...,6.150,19.45,0.623,179.55,132.11,165.75,162.16,1.765870e+09,2024-08-15,2024-07-15
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373.0,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",...,3.041,18.21,1.227,384.33,277.47,300.33,330.34,6.263840e+08,2024-08-15,2024-07-11


In [9]:
for ticker in sp500[26:50]:
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={TOKEN}'
    r = requests.get(url)
    data.loc[ticker] = r.json()

data.head()

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
MMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data.head(30)

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
MMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADBE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data['Beta'] = data['Beta'].astype(float)
data[data['Beta']>1]

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
MMM,MMM,Common Stock,3M Company,The 3M Company is an American multinational co...,66740,NYSE,USD,USA,LIFE SCIENCES,SURGICAL & MEDICAL INSTRUMENTS & APPARATUS,...,1.879,-9.08,1.001,105.52,68.63,101.29,97.98,553361000,2024-06-12,2024-05-23
ACN,ACN,Common Stock,Accenture plc,Accenture plc is an Irish-domiciled multinatio...,1467373,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",...,3.041,18.21,1.227,384.33,277.47,300.33,330.34,626384000,2024-08-15,2024-07-11
ADBE,ADBE,Common Stock,Adobe Systems Incorporated,Adobe Inc. is an American multinational comput...,796343,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,...,10.52,29.33,1.297,638.25,433.97,507.51,545.79,443400000,None,2005-03-24
AMD,AMD,Common Stock,Advanced Micro Devices Inc,"Advanced Micro Devices, Inc. (AMD) is an Ameri...",2488,NASDAQ,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,...,11.51,62.93,1.681,227.3,93.11,163.79,152.02,1616310000,None,1995-04-27
AAP,AAP,Common Stock,Advance Auto Parts Inc,"Advance Auto Parts, Inc. (Advance) is an Ameri...",1158449,NYSE,USD,USA,TRADE & SERVICES,RETAIL-AUTO & HOME SUPPLY STORES,...,0.682,18.06,1.226,87.92,47.19,65.63,65.36,59622000,2024-07-26,2024-07-12
AES,AES,Common Stock,The AES Corporation,The AES Corporation is a Fortune 500 company t...,874761,NYSE,USD,USA,ENERGY & TRANSPORTATION,COGENERATION SERVICES & SMALL POWER PRODUCERS,...,2.99,14.85,1.079,22.21,11.08,19.44,17.6,710667000,2024-08-15,2024-08-01
A,A,Common Stock,Agilent Technologies Inc,"Agilent Technologies, Inc. is an American anal...",1090872,NYSE,USD,USA,LIFE SCIENCES,INSTRUMENTS FOR MEAS & TESTING OF ELECTRICITY ...,...,6.24,24.88,1.049,155.06,96.3,136.74,132.82,291761000,2024-07-24,2024-07-02
ALK,ALK,Common Stock,Alaska Air Group Inc,Alaska Air Group is an airline holding company...,766421,NYSE,USD,USA,ENERGY & TRANSPORTATION,"AIR TRANSPORTATION, SCHEDULED",...,0.693,8.32,1.604,53.94,30.75,41.05,38.72,126475000,2020-03-05,2020-02-14
ALB,ALB,Common Stock,Albemarle Corp,Albemarle Corporation is a fine chemical manuf...,915913,NYSE,USD,USA,LIFE SCIENCES,"PLASTIC MATERIALS, SYNTH RESINS & NONVULCAN EL...",...,1.851,22.45,1.575,215.19,90.32,111.64,124.55,117527000,2024-07-01,2024-09-13
ARE,ARE,Common Stock,Alexandria Real Estate Equities Inc,"Alexandria Real Estate Equities, Inc. is an Am...",1035443,NYSE,USD,USA,REAL ESTATE & CONSTRUCTION,REAL ESTATE INVESTMENT TRUSTS,...,11.23,22.36,1.162,131.26,87.93,118.84,117.75,174883000,2024-07-15,2024-06-28


In [12]:
data[data['Beta']>1].groupby('Sector').count()

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Industry,Address,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
Sector,,,,,,,,,,,,,,,,,,,,,
ENERGY & TRANSPORTATION,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
LIFE SCIENCES,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
MANUFACTURING,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
REAL ESTATE & CONSTRUCTION,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
TECHNOLOGY,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
TRADE & SERVICES,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [13]:
data.to_csv('../data/sp500_Company_Overview.csv')